In [11]:
# Run order - 1
# Needed input files: 'second_genome_2.csv', 'data_discovery.csv'
# Generated output files: '_40_coefs.csv', 'top_11_mets.csv', 'coeff_validation.csv'

In [1]:
# Load libraries
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn import model_selection
import seaborn as sns
from string import ascii_letters
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV,LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
#Importing Data
second_genome=pd.DataFrame.from_csv('second_genome_2.csv')
second_genome.index=second_genome.index.astype('float64')
print (second_genome.shape)
discovery_mets=pd.DataFrame.from_csv('data_discovery.csv')
discovery_mets.index=discovery_mets.index.astype('float64')
print (discovery_mets.shape)

(540, 660)
(399, 665)


# Discovery Cohort Analysis

In [3]:
#Scale and standardize metabolites
X = discovery_mets[discovery_mets.columns[0:659]]
y = (discovery_mets['shannon'])
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
Xcolumns=X.columns
X = scaler.fit_transform(X)
X=pd.DataFrame(data=X,columns=Xcolumns)
print (X.shape)

(399, 659)


In [4]:
## run cross_val_score on ridge and lasso to get out-of-sample R2 scores across 10-CV
#defining L2 parameters to be tested
alphas = np.linspace(1,1000,200)
#Defining LASSO and Ridge parameters
lassocv=LassoCV(eps=0.175, n_alphas=200, alphas=None, fit_intercept=True, normalize=False, precompute='auto', cv=10)
ridgecv=RidgeCV(alphas=alphas,fit_intercept=True,normalize=False,cv=10)
#Running 10-fold CV score function to get mean out-of-sample R2
discovery_score=cross_val_score(lassocv,X,y,cv=10)
print ('mean out-of-sample R2 LASSO',np.mean(discovery_score))
discovery_score_ridge=cross_val_score(ridgecv,X,y,cv=10)
print ('mean out-of-sample R2 Ridge',np.mean(discovery_score_ridge))

mean out-of-sample R2 LASSO 0.44773547745248166
mean out-of-sample R2 Ridge 0.35023574199846985


In [5]:
#Run Cross-validation and extract Beta_coefficients for each model
#Save predictions from each test set
lassocv=LassoCV(eps=0.175, n_alphas=200, alphas=None, fit_intercept=True, normalize=False, precompute='auto', cv=10)
y=discovery_mets['shannon']
y=y.reset_index()
y.drop(['public_client_id'],1,inplace=True)
X_folds = np.array_split(X, 10)
y_folds = np.array_split(y, 10)
coefficients=pd.DataFrame(index=X.columns).astype('float64')
predictions=[]
alphas= []
score= []
for k in range(10):
    X_train = list(X_folds)
    X_test  = X_train.pop(k)
    X_train = np.concatenate(X_train)
    y_train = list(y_folds)
    y_test  = y_train.pop(k)
    y_test=[ x[0] for x in  list(y_test.values)]
    y_train = np.concatenate(y_train)
    lassocv.fit(X_train, y_train)
    predictions.append(lassocv.predict(X_test).flatten())
    coef=list(lassocv.coef_)
    coefficients[k]=coef
    alphas.append(lassocv.alpha_)
    score.append(r2_score(y_test,lassocv.predict(X_test)))
#The L1 penalty for each model
print (alphas)
predictions_lasso=[item for sublist in predictions for item in sublist]
#Checking r2 score and pearson r
print ('mean R2 Score LASSO',np.mean(score))
print ('std. deviation for R2 Score',np.std(score))
print ('S.E.M',np.std(score)/np.sqrt(10))
print ('observed v predicted pearson r',stats.pearsonr(discovery_mets['shannon'],predictions_lasso))

[0.039961541031946914, 0.04012354796331663, 0.0381683954022164, 0.041451457326540785, 0.03903085906919329, 0.0395668997409929, 0.0449161949755424, 0.03867589675958927, 0.03995461545690002, 0.041599615043147055]
mean R2 Score LASSO 0.44773547745248166
std. deviation for R2 Score 0.10613472897414773
S.E.M 0.0335627482402973
observed v predicted pearson r (0.6834440092504868, 3.207776674302782e-56)


In [6]:
#Identifying all metabolites with non-zero Beta Coefficients for figures 1B&C
for x in coefficients.index.tolist():
    if (coefficients.loc[x] == 0.0).sum()==10:
        coefficients.drop([x],inplace=True)
print (coefficients.shape)
#calculating mean beta-coefficient for each metabolite and counting no. of times each metabolite had a 0 beta-coefficient.
means=[]
std=[]
zeroes=[]
for x in coefficients.index.tolist():
    means.append((np.mean(coefficients.loc[x])))
    std.append((np.std(coefficients.loc[x])))
    zeroes.append((coefficients.loc[x] == 0.0).astype(int).sum())
coefficients['mean']=means
coefficients['std_dev']=std
coefficients['zeroes']=zeroes
#save table as csv
coefficients.to_csv('_40_coefs.csv')
coefficients.sort_values(by='mean',ascending=False).head()

(40, 10)


,0,1,2,3,4,5,6,7,8,9,mean,std_dev,zeroes
"root.metabolite.scaled.lipid.steroid.5alpha-androstan-3beta,17alpha-diol disulfate:scaled",0.066463,0.071688,0.075548,0.079401,0.086062,0.074617,0.079345,0.086107,0.080450,0.085282,0.078496,0.006176,0
root.metabolite.scaled.xenobiotics.benzoate_metabolism.hippurate:scaled,0.075347,0.058071,0.061615,0.059721,0.062048,0.061329,0.058928,0.060168,0.040132,0.037057,0.057441,0.010511,0
root.metabolite.scaled.xenobiotics.food_component_plant.cinnamoylglycine:scaled,0.047287,0.056804,0.021443,0.060724,0.050004,0.052995,0.086802,0.047123,0.069107,0.073827,0.056612,0.016886,0
root.metabolite.scaled.amino_acid.phenylalanine_and_tyrosine_metabolism.p-cresol sulfate:scaled,0.058071,0.066334,0.062401,0.050803,0.049018,0.056246,0.022637,0.050644,0.049763,0.050510,0.051643,0.011175,0
root.metabolite.scaled.xenobiotics.food_component_plant.methyl glucopyranoside (alpha + beta):scaled,0.020713,0.028351,0.036733,0.032002,0.023007,0.021079,0.025953,0.031032,0.034010,0.025521,0.027840,0.005232,0


In [7]:
#save top 11 metabolites to csv for classification analysis
coefficients[coefficients['zeroes']==0].to_csv('top_11_mets.csv')

# PD whole tree and Chao1 predictions

In [8]:
#Using all metabolites to predict PD whole tree and Chao1
lassocv=LassoCV(eps=0.175, n_alphas=200, alphas=None, fit_intercept=True, normalize=False, precompute='auto', cv=10)
discovery_PD_all=cross_val_score(lassocv,X,discovery_mets['PD_whole_tree'],cv=10)
print ('PD whole tree all 659 mets mean out-of-sample R2',np.mean(discovery_PD_all))
discovery_Chao_all=cross_val_score(lassocv,X,discovery_mets['chao1'],cv=10)
print ('Chao1 all 659 mets mean out-of-sample R2',np.mean(discovery_Chao_all))

PD whole tree all 659 mets mean out-of-sample R2 0.46595868693020304
Chao1 all 659 mets mean out-of-sample R2 0.33255103891132803


In [9]:
#Testing prediction of other diversity metrics using just the 40 mets identified.
W=pd.DataFrame()
for x in discovery_mets.columns.tolist():
    if x in coefficients.index.tolist():
        W[x]=X[x]
print (W.shape)
lassocv=LassoCV(eps=0.01, n_alphas=200, alphas=None, fit_intercept=True, normalize=False, precompute='auto', cv=10)
discovery_PD40_=cross_val_score(lassocv,W,discovery_mets['PD_whole_tree'],cv=10)
print ('PD whole tree 40 mets mean out-of-sample R2',np.mean(discovery_PD40_))
discovery_Chao40_=cross_val_score(lassocv,W,discovery_mets['chao1'],cv=10)
print ('Chao1 all 40 mean out-of-sample R2',np.mean(discovery_Chao40_))

(399, 40)
PD whole tree 40 mets mean out-of-sample R2 0.4992750699044487
Chao1 all 40 mean out-of-sample R2 0.36160743908467713


# Validation Cohort Analysis

In [12]:
#Metabolomics Validation
#Scaling and standardizing the validation cohort
y_validation = (second_genome['shannon'])
vendor = second_genome[second_genome.columns[0:659]]
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
Xcolumns=vendor.columns
Xindex=vendor.index
vendor = scaler.fit_transform(vendor)
vendor=pd.DataFrame(data=vendor,columns=Xcolumns)

In [13]:
#Run LASSO using all 659 Mets
## run cross_val_score on ridge and lasso to get out-of-sample R2 scores across 10-CV
lassocv=LassoCV(eps=0.175, n_alphas=200, alphas=None, fit_intercept=True, normalize=False, precompute='auto', cv=10)
validation_score=cross_val_score(lassocv,vendor,y_validation,cv=10)
print ('mean out-of-sample R2 LASSO',np.mean(validation_score))
print ('mean out-of-sample STD LASSO',np.std(validation_score))

mean out-of-sample R2 LASSO 0.3779993607744639
mean out-of-sample STD LASSO 0.06380484009950287


In [14]:
#Predict shannon using just the 40 metabolites identified in the discovery cohort
for x in vendor.columns.tolist():
    if x not in coefficients.index.tolist():
        vendor.drop([x],1,inplace=True)
print (vendor.shape)
lassocv=LassoCV(eps=0.05, n_alphas=200, alphas=None, fit_intercept=True, normalize=False, precompute='auto', cv=10)
validation40_score=cross_val_score(lassocv,vendor,y_validation,cv=10)
print ('mean out-of-sample R2 LASSO 40 mets validation',np.mean(validation40_score))
print ('std deviation of R2 score',np.std(validation40_score))

(540, 40)
mean out-of-sample R2 LASSO 40 mets validation 0.3425722058905879
std deviation of R2 score 0.07784909649313121


In [15]:
#Assessing whether performance is significantly different across the 10-CVs between whole metabolome model and the 40 metabolite model
print ('ttest 40 mets vs. 659 mets',stats.ttest_ind(validation_score,validation40_score))

ttest 40 mets vs. 659 mets Ttest_indResult(statistic=1.0558927573271495, pvalue=0.3049839359873753)


In [18]:
#Extracting Beta coefficients from 10-fold cv using only 40 mets in the validation set
lassocv=LassoCV(eps=0.05, n_alphas=200, alphas=None, fit_intercept=True, normalize=False, precompute='auto', cv=10)
y=y_validation
y=y.reset_index()
y.drop(['public_client_id'],1,inplace=True)
from sklearn.model_selection import KFold
X_folds = np.array_split(vendor, 10)
y_folds = np.array_split(y, 10)
coefficients_validation=pd.DataFrame(index=vendor.columns).astype('float64')
predictions_validation=[]
alphas= []
score_validation= []
for k in range(10):
    X_train = list(X_folds)
    X_test  = X_train.pop(k)
    X_train = np.concatenate(X_train)
    y_train = list(y_folds)
    y_test  = y_train.pop(k)
    y_test=[ x[0] for x in  list(y_test.values)]
    y_train = np.concatenate(y_train)
    lassocv.fit(X_train, y_train)
    predictions_validation.append(lassocv.predict(X_test).flatten())
    coef=list(lassocv.coef_)
    coefficients_validation[k]=coef
    alphas.append(lassocv.alpha_)
    score_validation.append(r2_score(y_test,lassocv.predict(X_test)))
print (lassocv.alpha_)
print (alphas)
predictions_validation=[item for sublist in predictions_validation for item in sublist]
#Identifying all metabolites with non-zero Beta Coefficients
means=[]
std=[]
zeroes=[]
for x in coefficients_validation.index.tolist():
    means.append((np.mean(coefficients_validation.loc[x])))
    std.append((np.std(coefficients_validation.loc[x])))
    zeroes.append((coefficients_validation.loc[x] == 0).astype(int).sum())
coefficients_validation['mean']=means
coefficients_validation['std_dev']=std
coefficients_validation['zeroes']=zeroes
coefficients_validation.sort_values(by='mean')
coefficients_validation.to_csv('coeff_validation.csv')

0.006481230778656312
[0.008129124361579266, 0.006676462334191416, 0.006490687317870491, 0.00675495415014577, 0.006788824190485644, 0.00671788522729803, 0.0069054870945478445, 0.006776899721933365, 0.006981219855903212, 0.006481230778656312]


In [19]:
#comparing beta coefficients across discovery and validation sets (Figure 6B)
top_11_=coefficients[coefficients['zeroes']==0].index.tolist()
correlating_coef=pd.DataFrame(index=top_11_)
correlating_coef['mean_discovery']=coefficients['mean']
correlating_coef['mean_validation']=coefficients_validation['mean']
correlating_coef['std_validation']=coefficients_validation['std_dev']
correlating_coef['std_discovery']=coefficients['std_dev']
#running pearson and spearman on the mean model beta-coefficients across cohorts
spearman=stats.spearmanr(correlating_coef['mean_discovery'],correlating_coef['mean_validation'])
print ('spearman rho=',spearman)
pearson=stats.pearsonr(correlating_coef['mean_discovery'],correlating_coef['mean_validation'])
print ('Pearson R=',pearson)

spearman rho= SpearmanrResult(correlation=0.9, pvalue=0.00015997142806871369)
Pearson R= (0.9374199588043084, 2.051161236736048e-05)
